In [1]:
import PyPDF2 as pypdf
import reportlab

In [20]:
#paper sizes
import reportlab.lib.pagesizes as pagesize

In [12]:
inputsig = pypdf.PdfFileReader('1-8.pdf')

In [13]:
inputsig.getPage(0)

RectangleObject([0, 0, 595.27559, 841.88976])

In [33]:
import tempfile
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

In [31]:
fp = tempfile.TemporaryFile()
can = canvas.Canvas(fp, pagesize=(2*595.27, 2*841.89))

In [32]:
can.drawString(100,100," Test Pages")

In [34]:
can.setStrokeColorRGB(0.2,0.5,0.3)
can.setFillColorRGB(1,0,1)
#an draw some lines
can.line(0,0,0,1.7*inch)
can.line(0,0,1*inch,0)
#an draw a rectangle
can.rect(0.2*inch,0.2*inch,1*inch,1.5*inch, fill=1)
#an make text go straight up
can.rotate(90)
#an change color
can.setFillColorRGB(0,0,0.77)
#an say hello (note after rotate the y coord needs to be negative!)
can.drawString(0.3*inch, -inch, "Hello World")

In [35]:
can.save()

In [36]:
twopdf = pypdf.PdfFileReader(fp)

In [37]:
genpdf = pypdf.PdfFileWriter()

In [47]:
inputpage = inputsig.getPage(1)
inputpage.add_transformation(pypdf.Transformation().rotate(90).translate(tx=10))

In [49]:
page = twopdf.getPage(0)
page.merge_page(inputpage)

In [50]:
genpdf.add_page(inputpage)
with open("./test.pdf", "wb") as f:
    genpdf.write(f)

In [ ]:
twopdf = pypdf.PdfFileReader(fp)

In [59]:
inputsig = pypdf.PdfFileReader('1-8.pdf')
inputpage = inputsig.getPage(0)
op = pypdf.Transformation().rotate(0).translate(ty=700, tx=500)
inputpage.addTransformation(op)
pagetwo = twopdf.getPage(0)
pagetwo.merge_page(inputpage)


genpdf = pypdf.PdfFileWriter()
genpdf.add_page(pagetwo)

with open("./copy_1-8.pdf", "wb") as f:
    genpdf.write(f)